### Import Libraries

In [21]:
import os
import glob
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [22]:
# Merge all CSV files in 'Opinions' into one DataFrame with mandatory 'content' field
opinions_dir = os.path.join(os.getcwd(), 'Opinions')
pattern = os.path.join(opinions_dir, '*.csv')
files = glob.glob(pattern)

dfs = []
for f in files:
    try:
        df = pd.read_csv(f, dtype=str, encoding='utf-8', engine='python')
    except Exception as e:
        print(f'Warning: failed to read {f}: {e}')
        continue

    # normalize column names
    df.columns = [str(c).strip().lower() for c in df.columns]

    # candidates
    opinion_candidates = ['opinion', 'content', 'text', 'review', 'body', 'comment']
    sentiment_candidates = ['sentiment', 'label', 'polarity']
    score_candidates = ['score', 'sentiment_score', 'compound', 'rating', 'rating_score']

    opinion_col = next((c for c in opinion_candidates if c in df.columns), None)
    sentiment_col = next((c for c in sentiment_candidates if c in df.columns), None)
    score_col = next((c for c in score_candidates if c in df.columns), None)

    # fallback: use first column as opinion if nothing matched
    if opinion_col is None and len(df.columns) > 0:
        opinion_col = df.columns[0]

    if opinion_col is None:
        # no usable columns in this file
        continue

    # build normalized frame with requested columns
    out = pd.DataFrame()
    out['opinion'] = df[opinion_col].astype(str).str.strip()
    out['sentiment'] = df[sentiment_col] if sentiment_col in df.columns else pd.NA
    out['score'] = df[score_col] if score_col in df.columns else pd.NA

    # drop empty opinions
    out = out[out['opinion'].notna() & (out['opinion'] != '')].copy()

    # coerce score to numeric where possible
    out['score'] = pd.to_numeric(out['score'], errors='coerce')

    dfs.append(out)

if dfs:
    consolidated = pd.concat(dfs, ignore_index=True)
else:
    consolidated = pd.DataFrame(columns=['opinion', 'sentiment', 'score'])
consolidated

,opinion,sentiment,score
0,enjoyable and smooth gameplay.;4;,<NA>,NaN
1,enjoyable and smooth gameplay.;4;,<NA>,NaN
2,enjoyable and smooth gameplay.;4;,<NA>,NaN
3,enjoyable and smooth gameplay.;4;,<NA>,NaN
4,enjoyable and smooth gameplay.;4;,<NA>,NaN
...,...,...,...
4795,Too complex for its own good. Late game lag is...,negative,2.0
4796,1945 runs at 1 frame per minute.,negative,2.0
4797,1000 hours in and I'm just getting started. Th...,positive,4.0
4798,"Game crashed, opened it again immediately.",positive,4.0
